In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import VECM
from statsmodels.stats.diagnostic import acorr_ljungbox

# Importación robusta de coint_johansen y kpss
try:
    from statsmodels.tsa.vector_ar.util import coint_johansen
except ImportError:
    try:
        from statsmodels.tsa.stattools import coint_johansen
    except ImportError:
        try:
            from statsmodels.tsa.vector_ar.vecm import coint_johansen
        except ImportError:
            print("Warning: coint_johansen not found. Will implement basic cointegration test.")
            coint_johansen = None

try:
    from statsmodels.tsa.stattools import kpss
except ImportError:
    print("Warning: kpss not available in this statsmodels version")
    kpss = None

import warnings
warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.style.use('default')
try:
    sns.set_palette("husl")
except:
    pass

# DataFrame indicadores
df_indicadores = pd.read_csv('../data/indicadores/indicadores_macroeconomicos.csv',
                             sep=';', encoding='latin1')
df_indicadores = df_indicadores.set_index('fecha')
# CONVERTIR EL ÍNDICE A STRING
df_indicadores.index = df_indicadores.index.astype(str)
df_indicadores.index.name = 'fecha'

# DataFrame termotrade  
df_termotrade = pd.read_csv('../data/indicadores/monthly_results_brazil.csv', 
                            sep=";", encoding='latin1')
df_termotrade = df_termotrade[['date', 'ctot_level']]
df_termotrade = df_termotrade.rename(columns={'ctot_level': 'termoftrade'})
df_termotrade = df_termotrade.set_index('date')
df_termotrade.index = pd.to_datetime(df_termotrade.index).strftime('%Y%m')
df_termotrade.index.name = 'fecha'


# Hacer el merge de los dos DataFrames
df_combined = df_indicadores.join(df_termotrade, how='outer')

# ==============================================================================
# AÑADIR VARIABLE DUMMY PARA LA PANDEMIA (Marzo 2020 - Septiembre 2020)
# ==============================================================================

# 1. Crear la columna 'pandemia_dummy' y llenarla con ceros
df_combined['pandemia_dummy'] = 0

# 2. Asignar el valor 1 para el período especificado
# Usamos .loc para seleccionar las filas por el índice (fecha)
start_date = '202003'
end_date = '202009'
df_combined.loc[start_date:end_date, 'pandemia_dummy'] = 1


# ==============================================================================
# AÑADIR VARIABLE PIB DE BRASIL
# ==============================================================================


gdp_brazil = pd.read_csv('../data/QGDP/brazil.csv', sep =",", encoding='latin1')
gdp_brazil = gdp_brazil.set_index(gdp_brazil.columns[0])
gdp_brazil.index = pd.to_datetime(gdp_brazil.index).strftime('%Y%m')  # String YYYYMM

gdp_brazil = gdp_brazil.rename(columns={gdp_brazil.columns[0]: 'GDP_BRAZIL'})

c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# PASO 1: CONSOLIDACIÓN DE DATOS (SIMPLIFICADO)
print("="*50)
print("🔍 PASO 1: CONSOLIDACIÓN DE DATOS")
print("="*50)
# Dataset final usando PIB como base (trimestral)
df_model = gdp_brazil.copy()
df_model = df_model.join(df_termotrade, how='inner')
df_model = df_model.join(df_indicadores, how='inner')

# NORMALIZACIÓN: PIB en logs, resto estandarizado (media=0, std=1)
print(f"\n🔄 APLICANDO TRANSFORMACIONES:")
print(f"  PIB Brasil: log transformation")
print(f"  Otros indicadores: normalización (μ=0, σ=1)")

# PIB en logs
df_model['log_gdp_brazil'] = np.log(df_model['GDP_BRAZIL'])

# Normalizar el resto de variables (excepto PIB)
variables_to_normalize = [col for col in df_model.columns if col != 'GDP_BRAZIL' and col != 'log_gdp_brazil']

for var in variables_to_normalize:
    mean_val = df_model[var].mean()
    std_val = df_model[var].std()
    df_model[f'{var}_norm'] = (df_model[var] - mean_val) / std_val
    print(f"  {var}: μ={mean_val:.3f} → 0, σ={std_val:.3f} → 1")

print(f"\n✅ Dataset consolidado: {df_model.shape}")
print(f"📅 Período: {df_model.index.min()}-{df_model.index.max()}")
print(f"🔢 Variables: {list(df_model.columns)}")
print("\n📊 Primeras observaciones:")
print(df_model.head())


🔍 PASO 1: CONSOLIDACIÓN DE DATOS

🔄 APLICANDO TRANSFORMACIONES:
  PIB Brasil: log transformation
  Otros indicadores: normalización (μ=0, σ=1)
  termoftrade: μ=100.592 → 0, σ=0.803 → 1
  PI_USA: μ=97.239 → 0, σ=5.035 → 1
  PI_FRA: μ=105.554 → 0, σ=6.657 → 1
  PI_GER: μ=96.813 → 0, σ=8.000 → 1
  PI_ITA: μ=105.058 → 0, σ=11.543 → 1
  PI_UK: μ=93.613 → 0, σ=7.838 → 1
  DGS10: μ=3.269 → 0, σ=1.290 → 1
  SPREAD_USA: μ=5.464 → 0, σ=2.609 → 1

✅ Dataset consolidado: (99, 18)
📅 Período: 200004-202410
🔢 Variables: ['GDP_BRAZIL', 'termoftrade', 'PI_USA', 'PI_FRA', 'PI_GER', 'PI_ITA', 'PI_UK', 'DGS10', 'SPREAD_USA', 'log_gdp_brazil', 'termoftrade_norm', 'PI_USA_norm', 'PI_FRA_norm', 'PI_GER_norm', 'PI_ITA_norm', 'PI_UK_norm', 'DGS10_norm', 'SPREAD_USA_norm']

📊 Primeras observaciones:
        GDP_BRAZIL  termoftrade   PI_USA  PI_FRA  PI_GER  PI_ITA  PI_UK  \
200004    194948.4    99.941465  92.6659   110.5    84.8   121.3   77.1   
200007    197616.2    99.846339  92.8373   111.7    86.4   121.0 

In [3]:
# PASO 2: VARIABLES DEL MODELO VECM (VERSIÓN CORREGIDA)
print("="*50)
print("📊 PASO 2: VARIABLES DEL MODELO (PONDERADO)")
print("="*50)

# Variables principales siguiendo metodología Talvi et al.
df_vecm = pd.DataFrame(index=df_model.index)

df_vecm['log_gdp_brazil'] = np.log(df_model['GDP_BRAZIL'])
df_vecm['log_tot_brazil'] = np.log(df_model['termoftrade'])  # posible quitar el logaritmo si no es necesario

# IP_G7 con ponderación específica
df_vecm['log_ip_g7'] = np.log(df_model['PI_USA'] * 0.40 + 
                              df_model['PI_FRA'] * 0.15 + 
                              df_model['PI_GER'] * 0.20 + 
                              df_model['PI_ITA'] * 0.12 + 
                              df_model['PI_UK'] * 0.13)

df_vecm['us_10y'] = df_model['DGS10']
df_vecm['risk_spread'] = df_model['SPREAD_USA']

# Limpiar datos
df_vecm = df_vecm.dropna()

print(f"✅ Variables creadas (IP_G7 ponderado): {list(df_vecm.columns)}")
print(f"📊 Observaciones: {len(df_vecm)}")
print(f"📅 Período final: {df_vecm.index.min()}-{df_vecm.index.max()}")
print(f"\n📈 Estadísticas descriptivas:")
print(df_vecm.describe().round(3))

📊 PASO 2: VARIABLES DEL MODELO (PONDERADO)
✅ Variables creadas (IP_G7 ponderado): ['log_gdp_brazil', 'log_tot_brazil', 'log_ip_g7', 'us_10y', 'risk_spread']
📊 Observaciones: 99
📅 Período final: 200004-202410

📈 Estadísticas descriptivas:
       log_gdp_brazil  log_tot_brazil  log_ip_g7  us_10y  risk_spread
count          99.000          99.000     99.000  99.000       99.000
mean           12.504           4.611      4.593   3.269        5.464
std             0.161           0.008      0.048   1.290        2.609
min            12.180           4.595      4.327   0.624        2.740
25%            12.368           4.604      4.570   2.225        3.653
50%            12.581           4.608      4.600   3.388        4.663
75%            12.623           4.618      4.616   4.270        6.487
max            12.729           4.634      4.672   6.054       16.788


In [4]:
# PASO 3: TESTS DE RAÍCES UNITARIAS
print("="*50)
print("🔍 PASO 3: TESTS ECONOMÉTRICOS")
print("="*50)

def test_unit_root(series, name):
    """Test ADF para raíces unitarias"""
    try:
        adf_result = adfuller(series.dropna())
        adf_stat = adf_result[0]
        adf_pval = adf_result[1]
        result = "No Estacionaria" if adf_pval > 0.05 else "Estacionaria"
        return {
            'variable': name, 
            'adf_stat': adf_stat, 
            'p_value': adf_pval, 
            'result': result
        }
    except Exception as e:
        print(f"Error en {name}: {e}")
        return {
            'variable': name, 
            'adf_stat': np.nan, 
            'p_value': np.nan, 
            'result': 'Error'
        }

# Tests en niveles
print("📊 TESTS DE RAÍCES UNITARIAS (NIVELES):")
unit_root_results = []
for col in df_vecm.columns:
    result = test_unit_root(df_vecm[col], col)
    unit_root_results.append(result)
    print(f"{col:<20}: p-val={result['p_value']:.4f} -> {result['result']}")

# Tests en primeras diferencias
print("\n📊 TESTS EN PRIMERAS DIFERENCIAS:")
diff_results = []
for col in df_vecm.columns:
    diff_series = df_vecm[col].diff().dropna()
    result = test_unit_root(diff_series, f"d_{col}")
    diff_results.append(result)
    print(f"d_{col:<19}: p-val={result['p_value']:.4f} -> {result['result']}")

# Tabla resumen corregida
print("\n📋 RESUMEN DE ESTACIONARIEDAD:")
print("-" * 70)
print(f"{'Variable':<20} {'Nivel p-val':<12} {'Diff p-val':<12} {'Orden':<8}")
print("-" * 70)

for i, col in enumerate(df_vecm.columns):
    nivel_pval = unit_root_results[i]['p_value']
    diff_pval = diff_results[i]['p_value']
    
    # Determinar orden de integración
    if not np.isnan(nivel_pval) and not np.isnan(diff_pval):
        if nivel_pval < 0.05:
            orden = "I(0)"
        elif diff_pval < 0.05:
            orden = "I(1)"
        else:
            orden = "I(2)+"
    else:
        orden = "Error"
    
    print(f"{col:<20} {nivel_pval:<12.4f} {diff_pval:<12.4f} {orden:<8}")

# Verificar condiciones para VECM
print("\n🔍 VERIFICACIÓN PARA VECM:")
i1_count = sum(1 for i in range(len(df_vecm.columns)) 
               if not np.isnan(unit_root_results[i]['p_value']) and 
                  not np.isnan(diff_results[i]['p_value']) and
                  unit_root_results[i]['p_value'] > 0.05 and 
                  diff_results[i]['p_value'] < 0.05)

print(f"Variables I(1): {i1_count}/{len(df_vecm.columns)}")
if i1_count >= 2:
    print("✅ Condiciones cumplidas para VECM (≥2 variables I(1))")
else:
    print("❌ Condiciones NO cumplidas para VECM")

🔍 PASO 3: TESTS ECONOMÉTRICOS
📊 TESTS DE RAÍCES UNITARIAS (NIVELES):
log_gdp_brazil      : p-val=0.4997 -> No Estacionaria
log_tot_brazil      : p-val=0.1235 -> No Estacionaria
log_ip_g7           : p-val=0.0000 -> Estacionaria
us_10y              : p-val=0.1668 -> No Estacionaria
risk_spread         : p-val=0.0106 -> Estacionaria

📊 TESTS EN PRIMERAS DIFERENCIAS:
d_log_gdp_brazil     : p-val=0.0000 -> Estacionaria
d_log_tot_brazil     : p-val=0.0000 -> Estacionaria
d_log_ip_g7          : p-val=0.0000 -> Estacionaria
d_us_10y             : p-val=0.0000 -> Estacionaria
d_risk_spread        : p-val=0.0000 -> Estacionaria

📋 RESUMEN DE ESTACIONARIEDAD:
----------------------------------------------------------------------
Variable             Nivel p-val  Diff p-val   Orden   
----------------------------------------------------------------------
log_gdp_brazil       0.4997       0.0000       I(1)    
log_tot_brazil       0.1235       0.0000       I(1)    
log_ip_g7            0.0000     

In [5]:
# PASO 3B: TESTS ADICIONALES DE RAÍZ UNITARIA
print("="*50)
print("🔍 PASO 3B: TESTS ADICIONALES (ROBUSTEZ)")
print("="*50)

# Test ADF con diferentes especificaciones
def test_adf_multiple(series, name):
    """Test ADF con diferentes especificaciones"""
    specs = ['n', 'c', 'ct']  # none, constant, constant+trend
    spec_names = ['Sin const', 'Con const', 'Const+trend']
    
    print(f"\n📊 {name}:")
    for i, spec in enumerate(specs):
        try:
            result = adfuller(series.dropna(), regression=spec)
            stat, pval = result[0], result[1]
            decision = "I(0)" if pval < 0.05 else "I(1)"
            print(f"  {spec_names[i]:<12}: stat={stat:>8.3f}, p-val={pval:>7.4f} -> {decision}")
        except:
            print(f"  {spec_names[i]:<12}: Error")

# Test KPSS (H0: Estacionaria - complementario al ADF)
def test_kpss_robust(series, name):
    """Test KPSS robusto"""
    if kpss is not None:
        try:
            # Con constante
            kpss_c = kpss(series.dropna(), regression='c')
            decision_c = "I(0)" if kpss_c[1] > 0.05 else "I(1)"
            
            # Con constante y tendencia  
            kpss_ct = kpss(series.dropna(), regression='ct')
            decision_ct = "I(0)" if kpss_ct[1] > 0.05 else "I(1)"
            
            print(f"  KPSS (const)  : stat={kpss_c[0]:>8.3f}, p-val={kpss_c[1]:>7.4f} -> {decision_c}")
            print(f"  KPSS (c+trend): stat={kpss_ct[0]:>8.3f}, p-val={kpss_ct[1]:>7.4f} -> {decision_ct}")
        except Exception as e:
            print(f"  KPSS: Error - {e}")

# Ejecutar tests robustos
for col in df_vecm.columns:
    test_adf_multiple(df_vecm[col], col)
    test_kpss_robust(df_vecm[col], col)
    print("-" * 50)

🔍 PASO 3B: TESTS ADICIONALES (ROBUSTEZ)

📊 log_gdp_brazil:
  Sin const   : stat=   3.306, p-val= 1.0000 -> I(1)
  Con const   : stat=  -1.568, p-val= 0.4997 -> I(1)
  Const+trend : stat=  -1.474, p-val= 0.8380 -> I(1)
  KPSS (const)  : stat=   1.519, p-val= 0.0100 -> I(1)
  KPSS (c+trend): stat=   0.372, p-val= 0.0100 -> I(1)
--------------------------------------------------

📊 log_tot_brazil:
  Sin const   : stat=   0.225, p-val= 0.7541 -> I(1)
  Con const   : stat=  -2.468, p-val= 0.1235 -> I(1)
  Const+trend : stat=  -2.663, p-val= 0.2516 -> I(1)
  KPSS (const)  : stat=   0.530, p-val= 0.0350 -> I(1)
  KPSS (c+trend): stat=   0.133, p-val= 0.0734 -> I(0)
--------------------------------------------------

📊 log_ip_g7:
  Sin const   : stat=   0.066, p-val= 0.7059 -> I(1)
  Con const   : stat=  -4.897, p-val= 0.0000 -> I(0)
  Const+trend : stat=  -5.070, p-val= 0.0002 -> I(0)
  KPSS (const)  : stat=   0.288, p-val= 0.1000 -> I(0)
  KPSS (c+trend): stat=   0.057, p-val= 0.1000 -> I(0)

In [7]:
# PASO 4: PREPARACIÓN PARA VECM Y TEST DE COINTEGRACIÓN
print("="*50)
print("🔍 PASO 4: COINTEGRACIÓN")
print("="*50)



# Usar todas las 5 variables del modelo original
vecm_vars_full = ['log_gdp_brazil', 'log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
df_vecm_full = df_vecm[vecm_vars_full].copy()

print(f"✅ Variables del modelo completo: {vecm_vars_full}")
print(f"📊 Observaciones: {len(df_vecm_full)}")

# Test de cointegración con 5 variables
if coint_johansen is not None:
    johansen_result = coint_johansen(df_vecm_full.values, det_order=1, k_ar_diff=2)
    print(f"\n📊 TEST JOHANSEN (5 variables):")
    print(f"Estadístico traza: {johansen_result.lr1}")
    print(f"Valores críticos 5%: {johansen_result.cvt[:, 1]}")
    n_coint = sum(johansen_result.lr1 > johansen_result.cvt[:, 1])
    print(f"✅ Relaciones de cointegración: {n_coint}")

print(f"\n🎯 Usar modelo de 5 variables (siguiendo Talvi)")

🔍 PASO 4: COINTEGRACIÓN
✅ Variables del modelo completo: ['log_gdp_brazil', 'log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
📊 Observaciones: 99

📊 TEST JOHANSEN (5 variables):
Estadístico traza: [89.18124676 54.76260563 25.77998298 11.37797131  1.21738178]
Valores críticos 5%: [79.3422 55.2459 35.0116 18.3985  3.8415]
✅ Relaciones de cointegración: 1

🎯 Usar modelo de 5 variables (siguiendo Talvi)


In [8]:

import pandas as pd
from statsmodels.tsa.api import VECM

# PASO 5: ESTIMACIÓN DEL MODELO VECM CON VARIABLE EXÓGENA
print("="*50)
print("🔍 PASO 5: ESTIMACIÓN VECM CON DUMMY EXÓGENA")
print("="*50)

# 1. Separar las variables endógenas de la exógena
# Las endógenas son las que se explican mutuamente en el sistema
endog_vars = ['log_gdp_brazil']
exog_vars = ['log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
df_endog = df_vecm_full[endog_vars]
df_exog = df_vecm_full[exog_vars]

# La exógena es la dummy. Usamos doble corchete para que sea un DataFrame.
df_dummy = df_combined[['pandemia_dummy']]


# --- PASO 4.5: PREPARACIÓN DE DATOS (VERSIÓN FINAL) ---
print("="*50)
print("🔧 PREPARANDO Y ALINEANDO DATOS")
print("="*50)

# 1. Preparar df_endog (Datos Trimestrales)
# Asumiendo que las fechas (ej. 200004) están en el índice.
# Lo convertimos a string y luego a fecha con el formato correcto.
df_endog.index = pd.to_datetime(df_endog.index.astype(str), format='%Y%m')
print("✅ Índice de df_endog (trimestral) convertido a Datetime.")

# 2. Preparar df_exog (Datos Mensuales)
# Hacemos exactamente lo mismo: asumimos que las fechas (ej. 200001) están en el índice.
df_exog.index = pd.to_datetime(df_exog.index.astype(str), format='%Y%m')
print("✅ Índice de df_exog (mensual) convertido a Datetime.")

df_dummy.index = pd.to_datetime(df_dummy.index.astype(str), format='%Y%m')
print("✅ Índice de df_dummy (mensual) convertido a Datetime.")

# 3. Convertir df_exog de Mensual a Trimestral
df_exog_quarterly = df_exog.resample('Q').max()
df_dummy_quarterly = df_dummy.resample('Q').max()
print("✅ Datos mensuales agregados a trimestres.")

# 4. Estandarizar Índices (Paso Clave para la Unión)
# Forzamos que ambos usen el primer día del trimestre como referencia.
df_endog.index = df_endog.index.to_period('Q').start_time
df_exog_quarterly.index = df_exog_quarterly.index.to_period('Q').start_time
df_dummy_quarterly.index = df_dummy_quarterly.index.to_period('Q').start_time
print("✅ Índices de ambos DataFrames estandarizados al inicio del trimestre.")

# 5. Unir los DataFrames y Limpiar
df_aligned = df_endog.join(df_exog_quarterly, how='inner')
df_aligned = df_aligned.join(df_dummy_quarterly, how='inner')
df_aligned.dropna(inplace=True)
print(f"📊 Datos alineados listos con {len(df_aligned)} observaciones.")

if not df_aligned.empty:
    print(f"📅 Nuevo período: {df_aligned.index.min().strftime('%Y-%m-%d')} a {df_aligned.index.max().strftime('%Y-%m-%d')}")
else:
    print("⚠️ ADVERTENCIA: El DataFrame sigue vacío. Revisa que los rangos de fechas se solapen.")


🔍 PASO 5: ESTIMACIÓN VECM CON DUMMY EXÓGENA
🔧 PREPARANDO Y ALINEANDO DATOS
✅ Índice de df_endog (trimestral) convertido a Datetime.
✅ Índice de df_exog (mensual) convertido a Datetime.
✅ Índice de df_dummy (mensual) convertido a Datetime.
✅ Datos mensuales agregados a trimestres.
✅ Índices de ambos DataFrames estandarizados al inicio del trimestre.
📊 Datos alineados listos con 99 observaciones.
📅 Nuevo período: 2000-04-01 a 2024-10-01


In [9]:
# SOLUCIÓN AL ERROR: "Only gave one variable to VECM"
# Dos opciones para implementar la metodología del BID

# =============================================================================
# OPCIÓN 1: MODELO DE CORRECCIÓN DE ERROR MANUAL (RECOMENDADO)
# Implementa exactamente lo que hace el BID
# =============================================================================

from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
import pandas as pd
import numpy as np

print("="*60)
print("🎯 IMPLEMENTANDO METODOLOGÍA BID - MODELO DE CORRECCIÓN DE ERROR")
print("="*60)

# Paso 1: Estimar relación de cointegración (largo plazo)
print("\n📊 PASO 1: ESTIMANDO RELACIÓN DE LARGO PLAZO")
print("-"*40)

# Variables para la relación de largo plazo (sin la dummy)
long_run_vars = ['log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
X_longrun = df_aligned[long_run_vars]
y_longrun = df_aligned['log_gdp_brazil']

# Agregar constante para la regresión de largo plazo
X_longrun_const = pd.concat([pd.Series(1, index=X_longrun.index, name='const'), X_longrun], axis=1)

# Estimar relación de largo plazo: PIB = β₀ + β₁*IP_G7 + β₂*TOT + β₃*US10Y + β₄*RISK + error
modelo_largo_plazo = OLS(y_longrun, X_longrun_const).fit()

print(f"✅ Relación de largo plazo estimada:")
print(f"   R² = {modelo_largo_plazo.rsquared:.4f}")
for i, (var, coef) in enumerate(zip(X_longrun_const.columns, modelo_largo_plazo.params)):
    print(f"   {var}: {coef:.4f} (p-valor: {modelo_largo_plazo.pvalues.iloc[i]:.4f})")

# Paso 2: Calcular término de corrección de error
print(f"\n🔧 PASO 2: CALCULANDO TÉRMINO DE CORRECCIÓN DE ERROR")
print("-"*40)

# Término de corrección de error = PIB_actual - PIB_equilibrio
# ECT_t = PIB_t - (β₀ + β₁*IP_G7_t + β₂*TOT_t + β₃*US10Y_t + β₄*RISK_t)
pib_equilibrio = modelo_largo_plazo.predict(X_longrun_const)
ect = y_longrun - pib_equilibrio
ect.name = 'ECT'

print(f"   Media del ECT: {ect.mean():.6f} (debe estar cerca de 0)")
print(f"   Desv. Estándar ECT: {ect.std():.4f}")

# Test de cointegración
coint_test = coint(y_longrun, X_longrun)
print(f"   Test de Cointegración (Engle-Granger): {coint_test[1]:.4f}")
if coint_test[1] < 0.05:
    print("   ✅ Evidencia de cointegración (p < 0.05)")
else:
    print("   ⚠️ Cointegración débil (p ≥ 0.05)")

# Paso 3: Estimar modelo de corrección de error (corto plazo)
print(f"\n📈 PASO 3: ESTIMANDO MODELO DE CORRECCIÓN DE ERROR")
print("-"*40)

# Preparar variables en diferencias
df_diff = df_aligned.diff().dropna()
df_diff.columns = [f'D_{col}' for col in df_diff.columns]

# El modelo de corrección de error:
# Δ(PIB_t) = α × ECT_{t-1} + γ₁×Δ(IP_G7_t) + γ₂×Δ(TOT_t) + γ₃×Δ(US10Y_t) + γ₄×Δ(RISK_t) + δ×PANDEMIA_t + ε_t

# Preparar datos para ECM
# ECT rezagado 1 período
ect_lagged = ect.shift(1)
ect_lagged.name = 'ECT_lag1'

# Variables explicativas del modelo de corto plazo
short_run_vars = ['D_log_ip_g7', 'D_log_tot_brazil', 'D_us_10y', 'D_risk_spread']
X_shortrun = df_diff[short_run_vars]

# Agregar ECT rezagado y dummy de pandemia
X_ecm = pd.concat([
    ect_lagged,
    X_shortrun,
    df_aligned['pandemia_dummy']
], axis=1).dropna()

# Variable dependiente: cambio en PIB
y_ecm = df_diff['D_log_gdp_brazil'].loc[X_ecm.index]

# Estimar modelo de corrección de error
modelo_ecm = OLS(y_ecm, X_ecm).fit()

print(f"✅ Modelo de Corrección de Error estimado:")
print(f"   R² = {modelo_ecm.rsquared:.4f}")
print(f"   Observaciones: {modelo_ecm.nobs}")

print(f"\n🔧 COEFICIENTES DEL MODELO:")
for var, coef, pval in zip(X_ecm.columns, modelo_ecm.params, modelo_ecm.pvalues):
    significance = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.10 else ""
    if var == 'ECT_lag1':
        print(f"   α (Corrección de Error): {coef:.4f} {significance}")
        if coef < 0:
            print(f"      ✅ Coeficiente negativo - Sistema estable")
            print(f"      📈 Velocidad de ajuste: {abs(coef)*100:.1f}% por trimestre")
        else:
            print(f"      ⚠️ Coeficiente positivo - Revisar especificación")
    elif var == 'pandemia_dummy':
        print(f"   δ (Efecto Pandemia): {coef:.4f} {significance}")
    else:
        print(f"   γ ({var}): {coef:.4f} {significance}")

print(f"\n📊 INTERPRETACIÓN:")
print(f"   - El modelo explica {modelo_ecm.rsquared*100:.1f}% de la variación en el crecimiento del PIB")
print(f"   - Efectos de largo plazo capturados en el término de corrección de error")
print(f"   - Efectos de corto plazo capturados en las variables en diferencias")

# Guardar resultados
resultados_bid = {
    'modelo_largo_plazo': modelo_largo_plazo,
    'modelo_ecm': modelo_ecm,
    'ect': ect,
    'r2_largo_plazo': modelo_largo_plazo.rsquared,
    'r2_corto_plazo': modelo_ecm.rsquared,
    'alpha': modelo_ecm.params['ECT_lag1'],
    'cointegration_pvalue': coint_test[1]
}

print(f"\n✅ Modelo BID implementado exitosamente - Resultados guardados en 'resultados_bid'")


🎯 IMPLEMENTANDO METODOLOGÍA BID - MODELO DE CORRECCIÓN DE ERROR

📊 PASO 1: ESTIMANDO RELACIÓN DE LARGO PLAZO
----------------------------------------
✅ Relación de largo plazo estimada:
   R² = 0.7066
   const: -27.8131 (p-valor: 0.0000)
   log_ip_g7: 0.6358 (p-valor: 0.0070)
   log_tot_brazil: 8.1661 (p-valor: 0.0000)
   us_10y: -0.0650 (p-valor: 0.0000)
   risk_spread: -0.0081 (p-valor: 0.0596)

🔧 PASO 2: CALCULANDO TÉRMINO DE CORRECCIÓN DE ERROR
----------------------------------------
   Media del ECT: -0.000000 (debe estar cerca de 0)
   Desv. Estándar ECT: 0.0873
   Test de Cointegración (Engle-Granger): 0.9927
   ⚠️ Cointegración débil (p ≥ 0.05)

📈 PASO 3: ESTIMANDO MODELO DE CORRECCIÓN DE ERROR
----------------------------------------
✅ Modelo de Corrección de Error estimado:
   R² = 0.5949
   Observaciones: 98.0

🔧 COEFICIENTES DEL MODELO:
   α (Corrección de Error): -0.0184 
      ✅ Coeficiente negativo - Sistema estable
      📈 Velocidad de ajuste: 1.8% por trimestre
   γ (

In [13]:
# =============================================================================
# MEJORAS Y DIAGNÓSTICOS PARA EL MODELO BID
# =============================================================================

import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import het_white, acorr_breusch_godfrey
import seaborn as sns

print("="*60)
print("🔧 DIAGNÓSTICOS Y MEJORAS DEL MODELO BID")
print("="*60)

# --- 1. VERIFICAR ESTACIONARIEDAD DE LAS SERIES ---
print("\n📊 1. TESTS DE RAÍZ UNITARIA (VERIFICAR I(1))")
print("-"*50)

def test_estacionariedad(serie, nombre):
    # Test ADF en niveles
    adf_niveles = adfuller(serie.dropna(), autolag='AIC')
    # Test ADF en diferencias
    adf_diff = adfuller(serie.diff().dropna(), autolag='AIC')
    
    print(f"{nombre}:")
    print(f"  Niveles: ADF={adf_niveles[0]:.3f}, p-valor={adf_niveles[1]:.3f} " + 
          ("(Estacionaria)" if adf_niveles[1] < 0.05 else "(No estacionaria)"))
    print(f"  Diferencias: ADF={adf_diff[0]:.3f}, p-valor={adf_diff[1]:.3f} " + 
          ("(Estacionaria)" if adf_diff[1] < 0.05 else "(No estacionaria)"))
    
    return adf_niveles[1] > 0.05 and adf_diff[1] < 0.05  # Es I(1)?

variables_test = ['log_gdp_brazil', 'log_ip_g7', 'log_tot_brazil', 'us_10y', 'risk_spread']
i1_variables = []

for var in variables_test:
    if var in df_aligned.columns:
        es_i1 = test_estacionariedad(df_aligned[var], var)
        if es_i1:
            i1_variables.append(var)

print(f"\n✅ Variables que son I(1): {i1_variables}")
print(f"ℹ️ Para cointegración, todas las variables deben ser I(1)")

# --- 2. TEST DE JOHANSEN (MÁS ROBUSTO) ---
print(f"\n🔗 2. TEST DE JOHANSEN PARA COINTEGRACIÓN")
print("-"*50)

from statsmodels.tsa.stattools import coint_johansen

# Preparar datos para Johansen (solo variables I(1))
if len(i1_variables) >= 2:
    datos_johansen = df_aligned[i1_variables].dropna()
    
    # Test de Johansen
    resultado_johansen = coint_johansen(datos_johansen.values, det_order=1, k_ar_diff=1)
    
    print(f"Variables incluidas: {i1_variables}")
    print(f"Observaciones: {len(datos_johansen)}")
    
    # Resultados del test de traza
    for i in range(len(i1_variables)):
        estadistico = resultado_johansen.lr1[i]
        valor_critico_5 = resultado_johansen.cvt[i, 1]
        print(f"H0: r≤{i} vs H1: r>{i}")
        print(f"  Estadístico: {estadistico:.3f}, Valor crítico 5%: {valor_critico_5:.3f}")
        if estadistico > valor_critico_5:
            print(f"  ✅ Rechaza H0 - Hay al menos {i+1} relación(es) de cointegración")
        else:
            print(f"  ❌ No rechaza H0 - Máximo {i} relación(es)")
            break
else:
    print("⚠️ No hay suficientes variables I(1) para test de Johansen")

# --- 3. DIAGNÓSTICOS DE RESIDUOS ---
print(f"\n🔍 3. DIAGNÓSTICOS DE RESIDUOS DEL ECM")
print("-"*50)

residuos_ecm = modelo_ecm.resid

# Test de normalidad
from scipy.stats import jarque_bera
jb_stat, jb_pvalue = jarque_bera(residuos_ecm)
print(f"Test Jarque-Bera (Normalidad): {jb_stat:.3f}, p-valor: {jb_pvalue:.3f}")

# Test de autocorrelación
try:
    bg_test = acorr_breusch_godfrey(modelo_ecm, nlags=4)
    print(f"Test Breusch-Godfrey (Autocorr): {bg_test[0]:.3f}, p-valor: {bg_test[1]:.3f}")
except:
    print("Test de autocorrelación no disponible")

# Test de heterocedasticidad
try:
    white_test = het_white(residuos_ecm, modelo_ecm.model.exog)
    print(f"Test White (Heteroced): {white_test[0]:.3f}, p-valor: {white_test[1]:.3f}")
except:
    print("Test de heterocedasticidad no disponible")

# --- 4. GRÁFICOS DE DIAGNÓSTICO ---
print(f"\n📈 4. GRÁFICOS DE DIAGNÓSTICO")
print("-"*50)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Diagnósticos del Modelo de Corrección de Error', fontsize=14)

# Residuos vs tiempo
axes[0,0].plot(residuos_ecm.index, residuos_ecm.values)
axes[0,0].axhline(y=0, color='r', linestyle='--')
axes[0,0].set_title('Residuos vs Tiempo')
axes[0,0].set_ylabel('Residuos')

# Q-Q plot
from scipy.stats import probplot
probplot(residuos_ecm, dist="norm", plot=axes[0,1])
axes[0,1].set_title('Q-Q Plot (Normalidad)')

# Histograma de residuos
axes[1,0].hist(residuos_ecm, bins=15, alpha=0.7, density=True)
axes[1,0].set_title('Histograma de Residuos')
axes[1,0].set_xlabel('Residuos')

# Residuos vs valores ajustados
valores_ajustados = modelo_ecm.fittedvalues
axes[1,1].scatter(valores_ajustados, residuos_ecm, alpha=0.6)
axes[1,1].axhline(y=0, color='r', linestyle='--')
axes[1,1].set_title('Residuos vs Valores Ajustados')
axes[1,1].set_xlabel('Valores Ajustados')
axes[1,1].set_ylabel('Residuos')

plt.tight_layout()
plt.show()

# --- 5. ANÁLISIS DEL TÉRMINO DE CORRECCIÓN DE ERROR ---
print(f"\n⚖️ 5. ANÁLISIS DEL TÉRMINO DE CORRECCIÓN DE ERROR")
print("-"*50)

# Gráfico del ECT
plt.figure(figsize=(12, 4))
plt.plot(ect.index, ect.values, linewidth=1.5)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.7)
plt.title('Término de Corrección de Error (ECT)')
plt.ylabel('Desviación del Equilibrio')
plt.xlabel('Tiempo')
plt.grid(True, alpha=0.3)
plt.show()

# Estadísticas del ECT
print(f"Estadísticas del ECT:")
print(f"  Media: {ect.mean():.6f}")
print(f"  Mediana: {ect.median():.4f}")
print(f"  Desv. Estándar: {ect.std():.4f}")
print(f"  Mín: {ect.min():.4f}, Máx: {ect.max():.4f}")

# --- 6. RECOMENDACIONES ---
print(f"\n💡 6. RECOMENDACIONES PARA MEJORAR EL MODELO")
print("-"*50)

print("Basado en los diagnósticos:")
print("1. ✅ El modelo tiene buen ajuste (R² = 59.5%)")
print("2. ✅ Coeficiente de corrección de error negativo (sistema estable)")
print("3. ⚠️ Ajuste lento (1.8% por trimestre) - normal en economías emergentes")

if jb_pvalue < 0.05:
    print("4. ⚠️ Residuos no normales - considerar outliers o cambio estructural")
else:
    print("4. ✅ Residuos aproximadamente normales")

print("\nSugerencias:")
print("• Considerar breaks estructurales en el período de análisis")
print("• Incluir más variables de control (términos cuadráticos, interacciones)")
print("• Verificar robustez con diferentes especificaciones del modelo")
print("• Analizar funciones impulso-respuesta para interpretación económica")

🔧 DIAGNÓSTICOS Y MEJORAS DEL MODELO BID

📊 1. TESTS DE RAÍZ UNITARIA (VERIFICAR I(1))
--------------------------------------------------
log_gdp_brazil:
  Niveles: ADF=-1.568, p-valor=0.500 (No estacionaria)
  Diferencias: ADF=-7.723, p-valor=0.000 (Estacionaria)
log_ip_g7:
  Niveles: ADF=-4.897, p-valor=0.000 (Estacionaria)
  Diferencias: ADF=-13.591, p-valor=0.000 (Estacionaria)
log_tot_brazil:
  Niveles: ADF=-2.468, p-valor=0.124 (No estacionaria)
  Diferencias: ADF=-11.339, p-valor=0.000 (Estacionaria)
us_10y:
  Niveles: ADF=-2.316, p-valor=0.167 (No estacionaria)
  Diferencias: ADF=-9.914, p-valor=0.000 (Estacionaria)
risk_spread:
  Niveles: ADF=-3.411, p-valor=0.011 (Estacionaria)
  Diferencias: ADF=-7.952, p-valor=0.000 (Estacionaria)

✅ Variables que son I(1): ['log_gdp_brazil', 'log_tot_brazil', 'us_10y']
ℹ️ Para cointegración, todas las variables deben ser I(1)

🔗 2. TEST DE JOHANSEN PARA COINTEGRACIÓN
--------------------------------------------------


ImportError: cannot import name 'coint_johansen' from 'statsmodels.tsa.stattools' (c:\Users\Usuario\anaconda3\envs\tftimeseriesII\lib\site-packages\statsmodels\tsa\stattools.py)

In [14]:
# =============================================================================
# MODELO BID CORREGIDO PARA INTEGRACIÓN MIXTA I(1)/I(0)
# =============================================================================


print("="*60)
print("🔧 MODELO BID CORREGIDO - INTEGRACIÓN MIXTA")
print("="*60)

# Basado en los tests de raíz unitaria:
# I(1): log_gdp_brazil, log_tot_brazil, us_10y
# I(0): log_ip_g7, risk_spread

# --- ESPECIFICACIÓN CORREGIDA ---
print("\n📋 ESPECIFICACIÓN DEL MODELO CORREGIDA:")
print("-"*40)

# Variables I(1) - pueden tener relación de cointegración
variables_i1 = ['log_gdp_brazil', 'log_tot_brazil', 'us_10y']
print(f"Variables I(1): {variables_i1}")

# Variables I(0) - entran en niveles como regresores adicionales
variables_i0 = ['log_ip_g7', 'risk_spread']  
print(f"Variables I(0): {variables_i0}")

# --- PASO 1: RELACIÓN DE LARGO PLAZO SOLO CON VARIABLES I(1) ---
print(f"\n🔗 PASO 1: COINTEGRACIÓN SOLO ENTRE VARIABLES I(1)")
print("-"*50)

from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS

# Relación de largo plazo: PIB_Brasil ~ TOT_Brasil + US_10Y
X_coint = df_aligned[['log_tot_brazil', 'us_10y']]
y_coint = df_aligned['log_gdp_brazil']

# Agregar constante
X_coint_const = pd.concat([pd.Series(1, index=X_coint.index, name='const'), X_coint], axis=1)

# Estimar relación de cointegración
modelo_coint = OLS(y_coint, X_coint_const).fit()
print(f"Relación de Cointegración (solo I(1)):")
print(f"  R² = {modelo_coint.rsquared:.4f}")

for var, coef, pval in zip(X_coint_const.columns, modelo_coint.params, modelo_coint.pvalues):
    sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.10 else ""
    print(f"  {var}: {coef:.4f} {sig} (p={pval:.4f})")

# Test de cointegración Engle-Granger
coint_test = coint(y_coint, X_coint)
print(f"\nTest Engle-Granger: p-valor = {coint_test[1]:.4f}")
if coint_test[1] < 0.05:
    print("  ✅ Evidencia de cointegración entre variables I(1)")
else:
    print("  ⚠️ Cointegración débil entre variables I(1)")

# Calcular ECT de la relación de cointegración
pib_equilibrio_coint = modelo_coint.predict(X_coint_const)
ect_coint = y_coint - pib_equilibrio_coint
ect_coint.name = 'ECT_I1'

# --- PASO 2: MODELO DE CORRECCIÓN DE ERROR AMPLIADO ---
print(f"\n📊 PASO 2: MODELO DE CORRECCIÓN DE ERROR AMPLIADO")
print("-"*50)

# Preparar variables
df_diff = df_aligned.diff().dropna()

# ECT rezagado (de la relación de cointegración I(1))
ect_lag1 = ect_coint.shift(1)
ect_lag1.name = 'ECT_lag1'

# Variables en diferencias (efectos de corto plazo de variables I(1))
diff_vars_i1 = ['log_tot_brazil', 'us_10y']  # Excluir PIB (variable dependiente)
X_diff_i1 = df_diff[[f'D_{var}' if f'D_{var}' in df_diff.columns else var for var in diff_vars_i1]]
X_diff_i1.columns = [f'D_{var}' for var in diff_vars_i1]

# Variables I(0) en niveles (efectos contemporáneos)
X_i0_levels = df_aligned[variables_i0]

# Variable dependiente: cambio en PIB
y_diff = df_diff['log_gdp_brazil']
y_diff.name = 'D_log_gdp_brazil'

# Combinar todas las variables explicativas
X_ecm_ampliado = pd.concat([
    ect_lag1,                              # Corrección de error I(1)
    X_diff_i1,                             # Diferencias de variables I(1)  
    X_i0_levels,                           # Niveles de variables I(0)
    df_aligned['pandemia_dummy']           # Variable dummy
], axis=1).dropna()

# Alinear variable dependiente
y_ecm_ampliado = y_diff.loc[X_ecm_ampliado.index]

# Verificar datos
print(f"Observaciones para ECM ampliado: {len(X_ecm_ampliado)}")
print(f"Variables explicativas: {list(X_ecm_ampliado.columns)}")

# --- PASO 3: ESTIMACIÓN DEL MODELO AMPLIADO ---
print(f"\n🎯 PASO 3: ESTIMACIÓN DEL ECM AMPLIADO")
print("-"*50)

# Estimar modelo
modelo_ecm_ampliado = OLS(y_ecm_ampliado, X_ecm_ampliado).fit()

print(f"✅ Modelo de Corrección de Error Ampliado:")
print(f"   R² = {modelo_ecm_ampliado.rsquared:.4f}")
print(f"   R² Ajustado = {modelo_ecm_ampliado.rsquared_adj:.4f}")
print(f"   Observaciones: {modelo_ecm_ampliado.nobs}")
print(f"   F-statistic: {modelo_ecm_ampliado.fvalue:.2f} (p={modelo_ecm_ampliado.f_pvalue:.4f})")

print(f"\n🔧 COEFICIENTES DEL MODELO AMPLIADO:")
for var, coef, pval, stderr in zip(X_ecm_ampliado.columns, 
                                  modelo_ecm_ampliado.params,
                                  modelo_ecm_ampliado.pvalues,
                                  modelo_ecm_ampliado.bse):
    sig = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.10 else ""
    
    if var == 'ECT_lag1':
        print(f"   α (Corrección Error I(1)): {coef:.4f} {sig} (se={stderr:.4f})")
        if coef < 0:
            print(f"      ✅ Negativo - Ajuste hacia equilibrio de largo plazo")
            print(f"      📈 Velocidad: {abs(coef)*100:.1f}% por trimestre")
    elif var.startswith('D_'):
        print(f"   γ ({var}): {coef:.4f} {sig} (se={stderr:.4f}) [Efecto corto plazo]")
    elif var in variables_i0:
        print(f"   β ({var}): {coef:.4f} {sig} (se={stderr:.4f}) [Variable I(0)]")
    elif var == 'pandemia_dummy':
        print(f"   δ (Pandemia): {coef:.4f} {sig} (se={stderr:.4f})")

# --- PASO 4: COMPARACIÓN DE MODELOS ---
print(f"\n📊 PASO 4: COMPARACIÓN DE ESPECIFICACIONES")
print("-"*50)

print("MODELO ORIGINAL (todas las variables):")
print(f"  R² = {modelo_ecm.rsquared:.4f}")
print(f"  α (corrección error) = {modelo_ecm.params['ECT_lag1']:.4f}")

print(f"\nMODELO CORREGIDO (separando I(1)/I(0)):")
print(f"  R² = {modelo_ecm_ampliado.rsquared:.4f}")
print(f"  α (corrección error) = {modelo_ecm_ampliado.params['ECT_lag1']:.4f}")

# Mejora en el ajuste
mejora_r2 = modelo_ecm_ampliado.rsquared - modelo_ecm.rsquared
print(f"\n📈 Mejora en R²: {mejora_r2:+.4f}")

if modelo_ecm_ampliado.rsquared > modelo_ecm.rsquared:
    print("✅ El modelo corregido tiene mejor ajuste")
else:
    print("⚠️ El modelo original tenía mejor ajuste")

# --- PASO 5: INTERPRETACIÓN ECONÓMICA ---
print(f"\n💡 PASO 5: INTERPRETACIÓN ECONÓMICA CORREGIDA")
print("-"*50)

print("EFECTOS DE LARGO PLAZO (vía cointegración I(1)):")
if 'ECT_lag1' in modelo_ecm_ampliado.params.index:
    alpha = modelo_ecm_ampliado.params['ECT_lag1'] 
    if alpha < 0:
        print(f"  ✅ PIB se ajusta {abs(alpha)*100:.1f}% por trimestre hacia equilibrio")
        print(f"  📅 Semivida del shock: ~{np.log(0.5)/np.log(1+alpha):.1f} trimestres")

print(f"\nEFECTOS DE CORTO PLAZO:")
for var in X_ecm_ampliado.columns:
    if var.startswith('D_') and var in modelo_ecm_ampliado.params.index:
        coef = modelo_ecm_ampliado.params[var]
        print(f"  {var}: {coef:.4f} (impacto inmediato)")

print(f"\nEFECTOS CONTEMPORÁNEOS (variables I(0)):")
for var in variables_i0:
    if var in modelo_ecm_ampliado.params.index:
        coef = modelo_ecm_ampliado.params[var]
        print(f"  {var}: {coef:.4f} (efecto en niveles)")

# Guardar resultados corregidos
resultados_bid_corregido = {
    'modelo_coint': modelo_coint,
    'modelo_ecm_ampliado': modelo_ecm_ampliado,
    'ect_i1': ect_coint,
    'variables_i1': variables_i1,
    'variables_i0': variables_i0,
    'r2_coint': modelo_coint.rsquared,
    'r2_ecm': modelo_ecm_ampliado.rsquared,
    'alpha_corregido': modelo_ecm_ampliado.params['ECT_lag1'],
    'coint_pvalue': coint_test[1]
}

print(f"\n✅ Modelo BID corregido completado - Resultados en 'resultados_bid_corregido'")

🔧 MODELO BID CORREGIDO - INTEGRACIÓN MIXTA

📋 ESPECIFICACIÓN DEL MODELO CORREGIDA:
----------------------------------------
Variables I(1): ['log_gdp_brazil', 'log_tot_brazil', 'us_10y']
Variables I(0): ['log_ip_g7', 'risk_spread']

🔗 PASO 1: COINTEGRACIÓN SOLO ENTRE VARIABLES I(1)
--------------------------------------------------
Relación de Cointegración (solo I(1)):
  R² = 0.6278
  const: -28.4920 *** (p=0.0000)
  log_tot_brazil: 8.9371 *** (p=0.0000)
  us_10y: -0.0652 *** (p=0.0000)

Test Engle-Granger: p-valor = 0.6900
  ⚠️ Cointegración débil entre variables I(1)

📊 PASO 2: MODELO DE CORRECCIÓN DE ERROR AMPLIADO
--------------------------------------------------
Observaciones para ECM ampliado: 98
Variables explicativas: ['ECT_lag1', 'D_log_tot_brazil', 'D_us_10y', 'log_ip_g7', 'risk_spread', 'pandemia_dummy']

🎯 PASO 3: ESTIMACIÓN DEL ECM AMPLIADO
--------------------------------------------------
✅ Modelo de Corrección de Error Ampliado:
   R² = 0.3349
   R² Ajustado = 0.2915
